In [13]:
import os
print(os.getcwd())

e:\2024t2\archive\archive


In [1]:
import os
import glob
import cv2
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor

# Set the working directory
# os.chdir('./')

# Paths to the directories containing the violence and non-violence videos
PATH_violence = "./Real_Life_Violence_Dataset/Violence"
PATH_nonviolence = "./Real_Life_Violence_Dataset/NonViolence"

# Create directories to save the extracted frames if they don't exist
os.makedirs('./data/Violence', exist_ok=True)
os.makedirs('./data/NonViolence', exist_ok=True)

# Function to extract frames
def extract_frames(video_path, save_dir):
    for path in tqdm(glob.glob(video_path + '/*')):
        fname = os.path.basename(path).split('.')[0]  # Get the base name of the file without the extension
        vidcap = cv2.VideoCapture(path)  # Initialize video capture object
        success, image = vidcap.read()  # Read the first frame
        count = 0  # Initialize frame counter

        # Loop to read frames from the video
        while success:
            if count % 1000 == 0:  # Save every 100th frame
                # Save the frame as a JPEG file
                cv2.imwrite(f"{save_dir}/{fname}-{str(count).zfill(4)}.jpg", image)
            success, image = vidcap.read()  # Read the next frame
            count += 1  # Increment the frame counter

# Extract frames from violent and non-violent videos in parallel
with ThreadPoolExecutor(max_workers=2) as executor:
    future_violence = executor.submit(extract_frames, PATH_violence, './data/Violence')
    future_nonviolence = executor.submit(extract_frames, PATH_nonviolence, './data/NonViolence')

# Ensure all threads are completed
future_violence.result()
future_nonviolence.result()

100%|██████████| 951/951 [02:49<00:00,  5.60it/s]


In [10]:
import os
import numpy as np
import matplotlib
matplotlib.use("Agg")

# Import the necessary packages
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import AveragePooling2D, Dropout, Flatten, Dense, Input, LSTM, TimeDistributed
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import SGD
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from imutils import paths
from tqdm import tqdm
import matplotlib.pyplot as plt
import warnings
import pickle
import cv2
from concurrent.futures import ThreadPoolExecutor

# Suppress warnings
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings('ignore', category=DeprecationWarning)

# Argument dictionary
args = {
    "dataset": "archive/archive/data",
    "model": "E:/2024t2/violence_model.h5",
    "label-bin": "E:/2024t2/lb.pickle",
    "epochs": 10,
    "plot": "E:/2024t2/plot.png"
}

# Initialize the set of labels
LABELS = set(["Violence", "NonViolence"])

# Define file paths for preprocessed data
data_file = 'E:\2024t2\archive\archive\data.npy'
labels_file = 'E:\2024t2\archive\archive\labels.npy'

if os.path.exists(data_file) and os.path.exists(labels_file):
    # Load the preprocessed data
    print("[INFO] loading preprocessed data...")
    data = np.load(data_file, allow_pickle=True)
    labels = np.load(labels_file)
else:
    # Function to process a single image
    def process_image(imagePath):
        # Extract the class label from the filename
        label = imagePath.split(os.path.sep)[-2]
        if label not in LABELS:
            return None, None

        # Load the image, swap color channels, and resize it to be a fixed 224x224 pixels
        image = cv2.imread(imagePath)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = cv2.resize(image, (224, 224))

        return image, label

    # Grab the list of images in our dataset directory
    print('-' * 100)
    print("[INFO] loading images...")
    print('-' * 100)
    imagePaths = list(paths.list_images(args["dataset"]))
    data = []
    labels = []

    # Process images in batches to reduce memory usage
    batch_size = 1000
    for i in range(0, len(imagePaths), batch_size):
        batchPaths = imagePaths[i:i + batch_size]

        # Use ThreadPoolExecutor to process images in parallel
        with ThreadPoolExecutor(max_workers=os.cpu_count()) as executor:
            results = list(tqdm(executor.map(process_image, batchPaths), total=len(batchPaths)))

        # Filter out None values (if any)
        data.extend([r[0] for r in results if r[0] is not None])
        labels.extend([r[1] for r in results if r[1] is not None])

    # Group frames by video
    video_data = {}
    for image, label in zip(data, labels):
        if label not in video_data:
            video_data[label] = []
        video_data[label].append(image)

    # Prepare the data and labels for LSTM
    data = []
    labels = []
    for label in video_data:
        # Pad sequences to have the same length
        sequence = video_data[label]
        sequence_length = max(len(seq) for seq in video_data.values())  # Find the longest sequence
        sequence = np.pad(sequence, [(0, sequence_length - len(sequence)), (0, 0), (0, 0), (0, 0)], mode='constant')  # Pad shorter sequences
        data.append(sequence)
        labels.append(label)

    # Convert the data and labels to NumPy arrays
    data = np.array(data)
    labels = np.array(labels)

    # Binarize the labels
    lb = LabelBinarizer()
    labels = lb.fit_transform(labels)
    labels = to_categorical(labels)

    # Save the preprocessed data
    print("[INFO] saving preprocessed data...")
    np.save(data_file, data)
    np.save(labels_file, labels)

# Split the data into training and testing sets
(trainX, testX, trainY, testY) = train_test_split(data, labels, test_size=0.2, stratify=labels, random_state=42)

----------------------------------------------------------------------------------------------------
[INFO] loading images...
----------------------------------------------------------------------------------------------------


ValueError: y has 0 samples: array([], dtype=float64)

In [11]:
import os
import numpy as np
import matplotlib
matplotlib.use("Agg")

# Import the necessary packages
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import AveragePooling2D, Dropout, Flatten, Dense, Input, LSTM, TimeDistributed
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import SGD
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from imutils import paths
from tqdm import tqdm
import matplotlib.pyplot as plt
import warnings
import pickle
import cv2
from concurrent.futures import ThreadPoolExecutor

# Suppress warnings
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings('ignore', category=DeprecationWarning)

# Argument dictionary
args = {
    "dataset": "archive/archive/real life violence situations/Real Life Violence Dataset",
    "model": "E:/2024t2/violence_model.h5",
    "label-bin": "E:/2024t2/lb.pickle",
    "epochs": 10,
    "plot": "E:/2024t2/plot.png"
}

# Initialize the set of labels
LABELS = set(["Violence", "NonViolence"])

# Define file paths for preprocessed data
data_file = 'E:/2024t2/data.npy'
labels_file = 'E:/2024t2/labels.npy'

if os.path.exists(data_file) and os.path.exists(labels_file):
    # Load the preprocessed data
    print("[INFO] loading preprocessed data...")
    data = np.load(data_file, allow_pickle=True)
    labels = np.load(labels_file)
else:
    # Function to process a single image
    def process_image(imagePath):
        # Extract the class label from the filename
        label = imagePath.split(os.path.sep)[-2]
        if label not in LABELS:
            return None, None

        # Load the image, swap color channels, and resize it to be a fixed 224x224 pixels
        image = cv2.imread(imagePath)
        if image is None:
            return None, None

        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = cv2.resize(image, (224, 224))

        return image, label

    # Grab the list of images in our dataset directory
    print('-' * 100)
    print("[INFO] loading images...")
    print('-' * 100)
    imagePaths = list(paths.list_images(args["dataset"]))
    print(f"[DEBUG] Total image paths found: {len(imagePaths)}")
    data = []
    labels = []

    # Process images in batches to reduce memory usage
    batch_size = 1000
    for i in range(0, len(imagePaths), batch_size):
        batchPaths = imagePaths[i:i + batch_size]

        # Use ThreadPoolExecutor to process images in parallel
        with ThreadPoolExecutor(max_workers=os.cpu_count()) as executor:
            results = list(tqdm(executor.map(process_image, batchPaths), total=len(batchPaths)))

        # Filter out None values (if any)
        data.extend([r[0] for r in results if r[0] is not None])
        labels.extend([r[1] for r in results if r[1] is not None])

    print(f"[DEBUG] Total images processed: {len(data)}")
    print(f"[DEBUG] Total labels processed: {len(labels)}")

    if len(data) == 0 or len(labels) == 0:
        print("[ERROR] No valid data or labels found.")
        raise ValueError("No valid data or labels found. Please check your dataset.")

    # Group frames by video
    video_data = {}
    for image, label in zip(data, labels):
        if label not in video_data:
            video_data[label] = []
        video_data[label].append(image)

    print(f"[DEBUG] Video data: {len(video_data)} classes")

    # Prepare the data and labels for LSTM
    data = []
    labels = []
    for label in video_data:
        # Pad sequences to have the same length
        sequence = video_data[label]
        sequence_length = max(len(seq) for seq in video_data.values())  # Find the longest sequence
        sequence = np.pad(sequence, [(0, sequence_length - len(sequence)), (0, 0), (0, 0), (0, 0)], mode='constant')  # Pad shorter sequences
        data.append(sequence)
        labels.append(label)

    print(f"[DEBUG] Data after padding: {len(data)} sequences")
    print(f"[DEBUG] Labels after padding: {len(labels)} sequences")

    # Convert the data and labels to NumPy arrays
    data = np.array(data)
    labels = np.array(labels)

    # Binarize the labels
    lb = LabelBinarizer()
    labels = lb.fit_transform(labels)
    labels = to_categorical(labels)

    # Save the preprocessed data
    print("[INFO] saving preprocessed data...")
    np.save(data_file, data)
    np.save(labels_file, labels)

# Split the data into training and testing sets
(trainX, testX, trainY, testY) = train_test_split(data, labels, test_size=0.2, stratify=labels, random_state=42)
print(f"[DEBUG] trainX shape: {trainX.shape}")
print(f"[DEBUG] testX shape: {testX.shape}")
print(f"[DEBUG] trainY shape: {trainY.shape}")
print(f"[DEBUG] testY shape: {testY.shape}")

----------------------------------------------------------------------------------------------------
[INFO] loading images...
----------------------------------------------------------------------------------------------------
[DEBUG] Total image paths found: 0
[DEBUG] Total images processed: 0
[DEBUG] Total labels processed: 0
[ERROR] No valid data or labels found.


ValueError: No valid data or labels found. Please check your dataset.

In [17]:
import os
import glob
import numpy as np
from PIL import Image
import tensorflow as tf

# 定义数据集路径
data_dir = 'E:\\2024t2\\archive\\archive\\data'

# 定义暴力和非暴力文件夹路径
nonviolence_dir = os.path.join(data_dir, 'NonViolence')
violence_dir = os.path.join(data_dir, 'Violence')

# 定义图像大小和序列长度
img_height, img_width = 64, 64
sequence_length = 10

# 加载图像数据
def load_images_from_folder(folder):
    images = []
    for filename in glob.glob(os.path.join(folder, '*.jpg')):
        img = Image.open(filename)
        img = img.resize((img_width, img_height))
        img = np.array(img)
        images.append(img)
    return np.array(images)

# 加载数据集
nonviolence_images = load_images_from_folder(nonviolence_dir)
violence_images = load_images_from_folder(violence_dir)

# 创建标签
nonviolence_labels = np.zeros(len(nonviolence_images))
violence_labels = np.ones(len(violence_images))

# 合并数据和标签
X = np.concatenate((nonviolence_images, violence_images), axis=0)
y = np.concatenate((nonviolence_labels, violence_labels), axis=0)

# 打乱数据集
indices = np.arange(X.shape[0])
np.random.shuffle(indices)
X = X[indices]
y = y[indices]

# 规范化图像数据
X = X / 255.0

# 确保数据的数量是 sequence_length 的整数倍
num_samples = (len(X) // sequence_length) * sequence_length
X = X[:num_samples]
y = y[:num_samples]

# 将数据划分为训练集和测试集
train_size = int(0.8 * len(X))
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# 确保训练集和测试集的大小也适应 sequence_length 的整数倍
train_samples = (len(X_train) // sequence_length) * sequence_length
test_samples = (len(X_test) // sequence_length) * sequence_length

X_train = X_train[:train_samples].reshape(-1, sequence_length, img_height, img_width, 3)
X_test = X_test[:test_samples].reshape(-1, sequence_length, img_height, img_width, 3)

# 由于我们是处理时间序列问题，我们需要在每个时间序列上分配一个标签
# 我们使用序列中的第一个标签作为序列的标签
y_train = y_train[:train_samples:sequence_length]
y_test = y_test[:test_samples:sequence_length]

# 构建模型
input_shape = (sequence_length, img_height, img_width, 3)

# 输入层
input_layer = tf.keras.Input(shape=input_shape)

# TimeDistributed的卷积层
cnn_layer = tf.keras.layers.TimeDistributed(tf.keras.layers.Conv2D(32, (3, 3), activation='relu'))(input_layer)
cnn_layer = tf.keras.layers.TimeDistributed(tf.keras.layers.MaxPooling2D((2, 2)))(cnn_layer)
cnn_layer = tf.keras.layers.TimeDistributed(tf.keras.layers.Flatten())(cnn_layer)

# LSTM层
lstm_layer = tf.keras.layers.LSTM(64)(cnn_layer)

# 全连接层
dense_layer = tf.keras.layers.Dense(64, activation='relu')(lstm_layer)
dropout_layer = tf.keras.layers.Dropout(0.5)(dense_layer)

# 输出层
output_layer = tf.keras.layers.Dense(1, activation='sigmoid')(dropout_layer)

# 定义模型
model = tf.keras.Model(inputs=input_layer, outputs=output_layer)

# 编译模型
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# 打印模型摘要
model.summary()

# 训练模型
model.fit(X_train, y_train, epochs=10, batch_size=16, validation_data=(X_test, y_test))

# 评估模型
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Loss: {loss}, Test Accuracy: {accuracy}')


Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 10, 64, 64, 3)]   0         
                                                                 
 time_distributed_6 (TimeDi  (None, 10, 62, 62, 32)    896       
 stributed)                                                      
                                                                 
 time_distributed_7 (TimeDi  (None, 10, 31, 31, 32)    0         
 stributed)                                                      
                                                                 
 time_distributed_8 (TimeDi  (None, 10, 30752)         0         
 stributed)                                                      
                                                                 
 lstm_2 (LSTM)               (None, 64)                7889152   
                                                           

In [18]:
import os
import glob
import numpy as np
from PIL import Image
import tensorflow as tf

# 定义数据集路径
data_dir = 'E:\\2024t2\\archive\\archive\\data'

# 定义暴力和非暴力文件夹路径
nonviolence_dir = os.path.join(data_dir, 'NonViolence')
violence_dir = os.path.join(data_dir, 'Violence')

# 定义图像大小和序列长度
img_height, img_width = 64, 64
sequence_length = 10

# 加载图像数据
def load_images_from_folder(folder):
    images = []
    for filename in glob.glob(os.path.join(folder, '*.jpg')):
        img = Image.open(filename)
        img = img.resize((img_width, img_height))
        img = np.array(img)
        images.append(img)
    return np.array(images)

# 加载数据集
nonviolence_images = load_images_from_folder(nonviolence_dir)
violence_images = load_images_from_folder(violence_dir)

# 创建标签
nonviolence_labels = np.zeros(len(nonviolence_images))
violence_labels = np.ones(len(violence_images))

# 合并数据和标签
X = np.concatenate((nonviolence_images, violence_images), axis=0)
y = np.concatenate((nonviolence_labels, violence_labels), axis=0)

# 打乱数据集
indices = np.arange(X.shape[0])
np.random.shuffle(indices)
X = X[indices]
y = y[indices]

# 规范化图像数据
X = X / 255.0

# 确保数据的数量是 sequence_length 的整数倍
num_samples = (len(X) // sequence_length) * sequence_length
X = X[:num_samples]
y = y[:num_samples]

# 将数据划分为训练集和测试集
train_size = int(0.8 * len(X))
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# 确保训练集和测试集的大小也适应 sequence_length 的整数倍
train_samples = (len(X_train) // sequence_length) * sequence_length
test_samples = (len(X_test) // sequence_length) * sequence_length

X_train = X_train[:train_samples].reshape(-1, sequence_length, img_height, img_width, 3)
X_test = X_test[:test_samples].reshape(-1, sequence_length, img_height, img_width, 3)

# 由于我们是处理时间序列问题，我们需要在每个时间序列上分配一个标签
# 我们使用序列中的第一个标签作为序列的标签
y_train = y_train[:train_samples:sequence_length]
y_test = y_test[:test_samples:sequence_length]

# 构建模型
input_shape = (sequence_length, img_height, img_width, 3)

# 输入层
input_layer = tf.keras.Input(shape=input_shape)

# TimeDistributed的卷积层
cnn_layer = tf.keras.layers.TimeDistributed(tf.keras.layers.Conv2D(32, (3, 3), activation='relu'))(input_layer)
cnn_layer = tf.keras.layers.TimeDistributed(tf.keras.layers.MaxPooling2D((2, 2)))(cnn_layer)
cnn_layer = tf.keras.layers.TimeDistributed(tf.keras.layers.Flatten())(cnn_layer)

# LSTM层
lstm_layer = tf.keras.layers.LSTM(64)(cnn_layer)

# 全连接层
dense_layer = tf.keras.layers.Dense(64, activation='relu')(lstm_layer)
dropout_layer = tf.keras.layers.Dropout(0.5)(dense_layer)

# 输出层
output_layer = tf.keras.layers.Dense(1, activation='sigmoid')(dropout_layer)

# 定义模型
model = tf.keras.Model(inputs=input_layer, outputs=output_layer)

# 编译模型
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# 打印模型摘要
model.summary()

# 定义早停回调
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', 
    patience=3, 
    restore_best_weights=True
)

# 训练模型
model.fit(
    X_train, 
    y_train, 
    epochs=50,  # 设置一个较大的值，实际训练的epoch数由早停决定
    batch_size=16, 
    validation_data=(X_test, y_test),
    callbacks=[early_stopping]
)

# 评估模型
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Loss: {loss}, Test Accuracy: {accuracy}')


Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 10, 64, 64, 3)]   0         
                                                                 
 time_distributed_9 (TimeDi  (None, 10, 62, 62, 32)    896       
 stributed)                                                      
                                                                 
 time_distributed_10 (TimeD  (None, 10, 31, 31, 32)    0         
 istributed)                                                     
                                                                 
 time_distributed_11 (TimeD  (None, 10, 30752)         0         
 istributed)                                                     
                                                                 
 lstm_3 (LSTM)               (None, 64)                7889152   
                                                           